<a href="https://colab.research.google.com/github/FadQode/teks-emo-analysis/blob/main/roberta_can_read_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# roBERTa Model Training with Own Dataset

In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git clone https://github.com/FadQode/teks-emo-analysis

Cloning into 'teks-emo-analysis'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 3), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 397.94 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW, get_scheduler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
LABEL_MAPPING = {
    'anger': 4,
    'fear': 5,
    'joy': 2,
    'love': 3,
    'sadness': 0,
    'surprise': 1
}


MODEL_NAME = "roberta-base"
BATCH_SIZE = 32
EPOCHS = 20
MAX_LENGTH = 256
LEARNING_RATE = 1e-5

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels.values.astype(int)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_data = pd.read_csv("/content/teks-emo-analysis/dataset/emotion_train.csv")
val_data = pd.read_csv("/content/teks-emo-analysis/dataset/emotion_val.csv")
test_data = pd.read_csv("/content/teks-emo-analysis/dataset/emotion_test.csv")

In [ ]:
# Check for missing labels in your train_data
print(train_data["emotion"].isnull().sum())  # Check if there are NaNs

train_data.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6784 entries, 0 to 6783
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     6784 non-null   object
 1   emotion  6784 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 106.1+ KB


In [ ]:
print(train_data['emotion'].value_counts())

emotion
1    1651
3    1118
4    1108
0    1016
5     962
2     929
Name: count, dtype: int64


In [ ]:
print("Label range in train data:", train_data['emotion'].min(), "-", train_data['emotion'].max())

Label range in train data: 0 - 5


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
train_dataset = EmotionDataset(train_data['text'], train_data['emotion'], tokenizer)
val_dataset = EmotionDataset(val_data['text'], val_data['emotion'], tokenizer)
test_dataset = EmotionDataset(test_data['text'], test_data['emotion'], tokenizer)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_MAPPING))
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = len(train_loader) * EPOCHS
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train_model():
    model.train()
    total_loss, total_correct = 0, 0
    for batch in tqdm(train_loader):
        batch = {key: val.to(device) for key, val in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["label"]
        )

        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        total_correct += (logits.argmax(dim=1) == batch["label"]).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / len(train_dataset)
    print(f"Training loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
def evaluate_model():
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            batch = {key: val.to(device) for key, val in batch.items()}

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["label"]
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            total_correct += (logits.argmax(dim=1) == batch["label"]).sum().item()

    avg_loss = total_loss / len(val_loader)
    accuracy = total_correct / len(val_dataset)
    print(f"Validation loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


In [ ]:
def evaluate_model():
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            logits = outputs.logits
            preds = logits.argmax(dim=1).cpu().numpy()
            labels = batch["label"].cpu().numpy()
            all_labels.extend(labels)
            all_preds.extend(preds)

    report = classification_report(all_labels, all_preds, target_names=list(LABEL_MAPPING.keys()))
    print(report)

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_model()
    evaluate_model()


Epoch 1/20


100%|██████████| 424/424 [02:30<00:00,  2.81it/s]


Training loss: 0.6841, Accuracy: 0.7619


100%|██████████| 97/97 [00:10<00:00,  9.18it/s]


Validation loss: 0.6555, Accuracy: 0.7828
Epoch 2/20


...

Validation loss: 0.7667, Accuracy: 0.8352
Epoch 20/20


100%|██████████| 424/424 [02:30<00:00,  2.82it/s]


Training loss: 0.0369, Accuracy: 0.9897


100%|██████████| 97/97 [00:10<00:00,  9.13it/s]

Validation loss: 0.7667, Accuracy: 0.8352


## roBERTa Model Saving

In [ ]:
model.save_pretrained("./roberta_emotion_model")
tokenizer.save_pretrained("./roberta_emotion_model")

('./roberta_emotion_model/tokenizer_config.json',
 './roberta_emotion_model/special_tokens_map.json',
 './roberta_emotion_model/vocab.json',
 './roberta_emotion_model/merges.txt',
 './roberta_emotion_model/added_tokens.json')

In [ ]:
print(test_data["emotion"].isnull().sum())  # Should be 0
test_data.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     1547 non-null   object
 1   emotion  1547 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.3+ KB


## Test roBERTa trained model with antoher data


### Test Set

In [ ]:
def predict_emotion_batch(texts):
    """Predict emotions for a batch of texts."""
    encoding = tokenizer(
        texts,
        max_length=128,
        padding=True,  # Batch padding
        truncation=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to('cuda' if torch.cuda.is_available() else 'cpu')
    attention_mask = encoding["attention_mask"].to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

    return predictions


In [ ]:
all_texts = test_data['text'].tolist()
true_labels = test_data['emotion'].values

# Predict in batches to avoid memory issues
BATCH_SIZE = 16  # Can be adjusted depending on your hardware
predicted_labels = []

for i in range(0, len(all_texts), BATCH_SIZE):
    batch_texts = all_texts[i:i + BATCH_SIZE]
    batch_predictions = predict_emotion_batch(batch_texts)
    predicted_labels.extend(batch_predictions)

# Step 5: Calculate accuracy
accuracy = (np.array(predicted_labels) == true_labels).mean()
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 84.16%


### Input Teks

In [ ]:
MODEL_DIR = "roberta-base"
# Load the model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained(MODEL_DIR)
model_input = RobertaForSequenceClassification.from_pretrained(MODEL_DIR)
model_input.eval()  # Set to evaluation mode


LABEL_MAPPING = {
    0: 'sadness',
    1: 'surprise',
    2: 'joy',
    3: 'love',
    4: 'anger',
    5: 'fear'
}

def predict_emotion(diary_text):
    """Predicts the emotion for a given diary text."""
    encoding = tokenizer(
        diary_text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    with torch.no_grad():
        outputs = model_input(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        predicted_emotion = LABEL_MAPPING[predicted_class]

    return predicted_emotion


# Input a diary text and predict its emotion
diary_text = input("Enter your diary text: ")
predicted_emotion = predict_emotion(diary_text)
print(f"The predicted emotion is: {predicted_emotion}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter your diary text: Diary  Tanggal: 14 Desember 2024  Hari ini aku sedih, uangku tambah banyak.namun aku bingung mau menghabiskan bagaimana. kemudian temanku datang dan dipalak   End of Diary.
The predicted emotion is: sadness


In [ ]:
import shutil

# Specify the folder to zip
folder_to_zip = 'roberta_emotion_model'

# Create a zip file
shutil.make_archive('roberta_model', 'zip', folder_to_zip)

'/content/teks-emo-analysis/roberta_model.zip'

In [ ]:
!git rm -r roberta_emotion_model

rm 'roberta_emotion_model/merges.txt'
rm 'roberta_emotion_model/model.safetensors'
rm 'roberta_emotion_model/special_tokens_map.json'
rm 'roberta_emotion_model/tokenizer_config.json'
rm 'roberta_emotion_model/vocab.json'


In [ ]:
from google.colab import files

# Download the zip file
files.download('roberta_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!du -sh roberta_model.zip

405M	roberta_model.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
shutil.move('roberta_model.zip', '/content/drive/MyDrive/')

Mounted at /content/drive


'/content/drive/MyDrive/roberta_model.zip'

# Fine Tune Gemma with Counseling Dataset

In [22]:
%pip install -U -q datasets trl bitsandbytes transformers accelerate peft wandb gradio

In [25]:
!pip uninstall bitsandbytes

Found existing installation: bitsandbytes 0.45.0
Uninstalling bitsandbytes-0.45.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bitsandbytes-0.45.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bitsandbytes/*
    /usr/local/lib/python3.10/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/tests/slow/__init__.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_dpo_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_sft_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/testing_constants.py
    /usr/local/lib/python3.10/dist-packages/tests/test_best_of_n_sampler.py
    /usr/local/lib/python3.10/dist-packages/tests/test_cli.py
    /usr/local/lib/python3.10/dist-packages/tests/test_core.py
    /usr/local/lib/python3.10/dist-packages/tests/test_cpo_trainer.py
    /usr/local/lib/python3.10/dist-packages/tests/test_data_collator_completion_only.py
    /usr/local/lib/pyt

In [26]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install torch --upgrade

  Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl.metadata (2.9 kB)
Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl (69.1 MB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.2 requires transformers<4.47.0, but you have transformers 4.47.0 which is i

## LLM Setup and Paramater Tuning wih QLORA

### Library Import

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig

from google.colab import userdata
from huggingface_hub import login

### WandB and Hugging Face Setup

In [2]:
hf_token = userdata.get("HF-SECRET")
wb_token = userdata.get("WB-SECRET")

login(token=hf_token)
wandb.login(key=wb_token)

run = wandb.init(
    project="Fine-tune Gemma 2B on Mental-Console Chatbot Dataset",
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fadhileq28 (fadhileq28-universitas-muhammadiyah-surakarta). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### QLoRA Config

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

base_model = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Tokenizing and Inference Test

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [11]:
with open("/content/diary.txt") as d:
  diary = str(d.read())

messages = [
    {
        "role": "user",
        "content": diary
    }
]

prompt = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True)
inputs = tokenizer(prompt,
                   return_tensors="pt",
                   padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=800, num_return_sequences=1)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

text

'user\nPagi ini saya bangun agak lebih awal daripada biasanya. Biasanya, saya suka menunda-nunda, tapi entah kenapa tubuh saya tidak ingin berlama-lama di tempat tidur. Mungkin juga karena tidur semalam cukup nyenyak. Saya merasa segar, walaupun semalam banyak pikiran yang berlarian di kepala saya.\n\nSetelah bangun, saya mulai dengan rutinitas kecil, menyeduh kopi sambil membuka jendela untuk merasakan udara pagi yang dingin. Rasanya, udara pagi yang segar itu memberi saya energi baru untuk menghadapi hari. Saya duduk di meja makan, memandangi langit yang mulai terang, dan tiba-tiba muncul banyak pertanyaan di kepala saya. Apa yang ingin saya capai hari ini? Apa yang ingin saya buktikan pada diri sendiri? Saya tahu saya sering terlalu banyak berpikir, tetapi entah kenapa pagi ini perasaan itu datang begitu kuat.\n\nSeharian ini saya berusaha menjaga fokus. Ada banyak pekerjaan yang harus diselesaikan, dan meskipun terkadang saya merasa terjebak dalam rutinitas, hari ini saya coba meni

In [12]:
print(text.split("model")[1])


1. **Berikan perhatian pada diri Anda.** Penting untuk memahami bagaimana Anda merasa dan apa yang ingin Anda capai hari ini.
2. **Berikan makna dan tujuan kepada tugas-tugas Anda.** Ini akan membantu Anda untuk menemukan motivasi dan tujuan yang lebih besar dari hidup Anda.
3. **Berikan waktu untuk relaxasi dan hiburan.** Ini akan membantu Anda untuk menyeimbangkan aktivitas dan waktu untuk rest.
4. **Berikan penghargaan diri Anda.** Berikan penghargaan kepada diri Anda atas kemajuan Anda dan tunjukkanlah makna yang lebih besar dari hidup Anda.
5. **Berikan dukungan kepada diri Anda.** Berikan dukungan dan motivasi kepada diri Anda agar Anda dapat mengatasi tantangan dan mencapai tujuan Anda.


### PEFT(Parameter Efficient Fine-Tuning)

In [13]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): G

In [14]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules='all-linear',
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, peft_config)

In [15]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


## Counseling Dataset Preparation

In [25]:
import pandas as pd

df = pd.read_csv("hf://datasets/nbertagnolli/counsel-chat/20220401_counsel_chat.csv")
df = df[["questionTitle", "questionText", "answerText"]]
df.to_csv("fine tune data.csv", index=False)

In [18]:
def prepare_data(row):
    return f"Question Title: {row['questionTitle']}\nQuestion Text: {row['questionText']}\n\nAnswer: {row['answerText']}"

df['formatted_text'] = df.apply(prepare_data, axis=1)

# Save the formatted data as a text file
with open("formatted_dataset.txt", "w") as f:
    for text in df['formatted_text']:
        f.write(text + "\n\n")

In [22]:
dataset = load_dataset("text", data_files={"train": "formatted_dataset.txt"})
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15836
    })
})

In [31]:
dataset_name = "fine-tune-data.csv"
dataset = load_dataset(data_files={"train": "fine-tune-data.csv"})
dataset

TypeError: load_dataset() missing 1 required positional argument: 'path'

/content
